In [ ]:
from sqlalchemy import create_engine, Table, MetaData, insert, select
import os


DATABASE_URL = "mysql+pymysql://root:@localhost/csvdata01_02102024"
engine = create_engine(DATABASE_URL)

table_main = 'ratehawk'


def create_sql_file(output_file_path):
    """
    Creates an SQL file that contains a query to fetch data from `innova_hotels_main` table
    where SupplierCode is "Ratehawk".

    :param output_file_path: The path where the SQL file will be saved.
    """
    sql_query = "SELECT * FROM innova_hotels_main WHERE SupplierCode=\"Ratehawk\";"

    try:
        # Write the SQL query to the file
        with open(output_file_path, "w") as sql_file:
            sql_file.write(sql_query)

        print(f"SQL file successfully created at: {output_file_path}")
    except Exception as e:
        print(f"An error occurred while creating the SQL file: {e}")

# Define the output file path (change this to your desired path)
output_path = os.path.join(os.getcwd(), "ratehawk_data.sql")

# Create the SQL file
create_sql_file(output_path)


In [ ]:
from sqlalchemy import create_engine, Table, MetaData, insert, select
import os

# Database URL for connecting to MySQL
DATABASE_URL = "mysql+pymysql://root:@localhost/csvdata01_02102024"
engine = create_engine(DATABASE_URL)

def fetch_ratehawk_data():
    """
    Fetches data from the `innova_hotels_main` table where SupplierCode is "Ratehawk"
    and writes the data into a text file.
    """
    connection = engine.connect()
    metadata = MetaData()

    # Reflect the table structure from the database
    table_main = Table('innova_hotels_main', metadata, autoload_with=engine)

    # Create a SELECT query to fetch data
    query = select(table_main).where(table_main.c.SupplierCode == "Ratehawk")

    try:
        result = connection.execute(query).fetchall()
        print(f"Fetched {len(result)} records from the database.")

        # Save result to file
        output_file_path = os.path.join(os.getcwd(), "ratehawk_data.sql")
        with open(output_file_path, "w") as output_file:
            for row in result:
                output_file.write(str(row) + "\n")

        print(f"Ratehawk data saved to {output_file_path}")

    except Exception as e:
        print(f"An error occurred while fetching data: {e}")

    finally:
        connection.close()

fetch_ratehawk_data()


In [3]:
from sqlalchemy import create_engine, Table, MetaData, insert, select
import os
from dotenv import load_dotenv


load_dotenv()

# Database connection details
db_host = os.getenv('DB_HOST')
db_user = os.getenv('DB_USER')
db_pass = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')


# Database URL for connecting to local MySQL database
LOCAL_DATABASE_URL = "mysql+pymysql://root:@localhost/csvdata01_02102024"
local_engine = create_engine(LOCAL_DATABASE_URL)

# Database URL for connecting to server MySQL database
SERVER_DATABASE_URL = f"mysql+pymysql://{db_user}:{db_pass}@{db_host}/{db_name}"
server_engine = create_engine(SERVER_DATABASE_URL)


def fetch_and_insert_ratehawk_data():
    """
    Fetches data from the `innova_hotels_main` table in the local database where SupplierCode is "Ratehawk"
    and inserts the data into the same table on the server database.
    Displays progress immediately for each chunk inserted.
    """
    local_connection = local_engine.connect()
    server_connection = server_engine.connect()
    metadata = MetaData()

    # Reflect the table structure from the local database
    table_main = Table('innova_hotels_main', metadata, autoload_with=local_engine)

    # Create a SELECT query to fetch data
    query = select(table_main).where(table_main.c.SupplierCode == "Ratehawk")

    try:
        # Fetch data from the local database
        result = local_connection.execute(query).fetchall()
        print(f"Fetched {len(result)} records from the local database.")

        # Insert data into the server database in chunks
        if result:
            server_table = Table('innova_hotels_main', metadata, autoload_with=server_engine)
            CHUNK_SIZE = 1000
            for i in range(0, len(result), CHUNK_SIZE):
                chunk = result[i:i + CHUNK_SIZE]
                insert_query = insert(server_table).values([dict(row._mapping) for row in chunk])
                server_connection.execute(insert_query)
                print(f"Inserted {len(chunk)} records into the server database. Total inserted so far: {i + len(chunk)}")
        else:
            print("No data to insert.")

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        local_connection.close()
        server_connection.close()

fetch_and_insert_ratehawk_data()


Fetched 1713000 records from the local database.
Inserted 1000 records into the server database. Total inserted so far: 1000
Inserted 1000 records into the server database. Total inserted so far: 2000
Inserted 1000 records into the server database. Total inserted so far: 3000
Inserted 1000 records into the server database. Total inserted so far: 4000
Inserted 1000 records into the server database. Total inserted so far: 5000
Inserted 1000 records into the server database. Total inserted so far: 6000
Inserted 1000 records into the server database. Total inserted so far: 7000
Inserted 1000 records into the server database. Total inserted so far: 8000
Inserted 1000 records into the server database. Total inserted so far: 9000
Inserted 1000 records into the server database. Total inserted so far: 10000
Inserted 1000 records into the server database. Total inserted so far: 11000
Inserted 1000 records into the server database. Total inserted so far: 12000
Inserted 1000 records into the serve

KeyboardInterrupt: 

In [ ]:
from sqlalchemy import create_engine, Table, MetaData, insert, select
from sqlalchemy.exc import SQLAlchemyError
import os
from dotenv import load_dotenv


load_dotenv()

# Database connection details
db_host = os.getenv('DB_HOST')
db_user = os.getenv('DB_USER')
db_pass = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')


# Database URL for connecting to local MySQL database
LOCAL_DATABASE_URL = "mysql+pymysql://root:@localhost/csvdata01_02102024"
local_engine = create_engine(LOCAL_DATABASE_URL)

# Database URL for connecting to server MySQL database
SERVER_DATABASE_URL = f"mysql+pymysql://{db_user}:{db_pass}@{db_host}/{db_name}"
server_engine = create_engine(SERVER_DATABASE_URL)


def fetch_and_insert_ratehawk_data():
    """
    Fetches data from the `innova_hotels_main` table in the local database where SupplierCode is "Ratehawk"
    and inserts the data into the same table on the server database.
    Each chunk is committed immediately after insertion to ensure data persistence.
    """
    local_connection = local_engine.connect()
    server_connection = server_engine.connect()
    metadata = MetaData()

    # Reflect the table structure from the local database
    table_main = Table('innova_hotels_main', metadata, autoload_with=local_engine)

    # Create a SELECT query to fetch data
    query = select(table_main).where(table_main.c.SupplierCode == "Ratehawk")

    try:
        # Fetch data from the local database
        result = local_connection.execute(query).fetchall()
        print(f"Fetched {len(result)} records from the local database.")

        # Insert data into the server database in chunks
        if result:
            server_table = Table('innova_hotels_main', metadata, autoload_with=server_engine)
            CHUNK_SIZE = 1000
            total_inserted = 0

            for i in range(0, len(result), CHUNK_SIZE):
                chunk = result[i:i + CHUNK_SIZE]

                # Use a transaction for each chunk
                with server_engine.begin() as transaction:
                    try:
                        insert_query = insert(server_table).values([dict(row._mapping) for row in chunk])
                        transaction.execute(insert_query)
                        total_inserted += len(chunk)
                        print(f"Inserted {len(chunk)} records into the server database. Total inserted so far: {total_inserted}")
                    except SQLAlchemyError as e:
                        print(f"An error occurred while inserting chunk: {e}")

        else:
            print("No data to insert.")

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        local_connection.close()
        server_connection.close()

fetch_and_insert_ratehawk_data()


Fetched 1713000 records from the local database.
Inserted 1000 records into the server database. Total inserted so far: 1000
Inserted 1000 records into the server database. Total inserted so far: 2000
Inserted 1000 records into the server database. Total inserted so far: 3000
Inserted 1000 records into the server database. Total inserted so far: 4000
Inserted 1000 records into the server database. Total inserted so far: 5000
Inserted 1000 records into the server database. Total inserted so far: 6000
Inserted 1000 records into the server database. Total inserted so far: 7000
Inserted 1000 records into the server database. Total inserted so far: 8000
Inserted 1000 records into the server database. Total inserted so far: 9000
Inserted 1000 records into the server database. Total inserted so far: 10000
Inserted 1000 records into the server database. Total inserted so far: 11000
Inserted 1000 records into the server database. Total inserted so far: 12000
Inserted 1000 records into the serve

In [6]:
from sqlalchemy import inspect, create_engine
from sqlalchemy.exc import SQLAlchemyError
import os 
from dotenv import load_dotenv

load_dotenv()

# Database connection details
db_host = os.getenv('DB_HOST')
db_user = os.getenv('DB_USER')
db_pass = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')

SERVER_DATABASE_URL = f"mysql+pymysql://{db_user}:{db_pass}@{db_host}/{db_name}"
server_engine = create_engine(SERVER_DATABASE_URL)
table_1 = "vervotech_mapping"
table_2 = "innova_hotels_main"

def get_table_columns(engine, table_name):
    try:
        # Use the inspector to get table information
        inspector = inspect(engine)
        columns = inspector.get_columns(table_name)
        print(f"Columns in table '{table_name}':")
        for column in columns:
            print(f"- {column['name']} ({column['type']})")
    except SQLAlchemyError as e:
        print(f"Error inspecting table '{table_name}': {e}")

# Call the function to inspect the table
get_table_columns(server_engine, table_1)
print("\n")
get_table_columns(server_engine, table_2)


Columns in table 'vervotech_mapping':
- Id (INTEGER)
- last_update (VARCHAR(255))
- VervotechId (VARCHAR(255))
- UpdateDateFormat (VARCHAR(255))
- ProviderHotelId (VARCHAR(255))
- ProviderFamily (VARCHAR(255))
- ModifiedOn (VARCHAR(255))
- ChannelIds (VARCHAR(255))
- ProviderLocationCode (VARCHAR(255))
- status (VARCHAR(255))
- hotel_city (VARCHAR(255))
- hotel_name (VARCHAR(255))
- hotel_country (VARCHAR(255))
- hotel_longitude (VARCHAR(255))
- hotel_latitude (VARCHAR(255))
- country_code (VARCHAR(255))
- content_update_status (VARCHAR(255))
- created_at (TIMESTAMP)


Columns in table 'innova_hotels_main':
- Id (INTEGER)
- IttId (VARCHAR(100))
- VervotechId (VARCHAR(255))
- SupplierCode (VARCHAR(255))
- HotelId (VARCHAR(100))
- DestinationId (VARCHAR(255))
- City (VARCHAR(255))
- CityCode (VARCHAR(100))
- State (VARCHAR(100))
- StateCode (VARCHAR(100))
- PostCode (VARCHAR(100))
- Country (VARCHAR(255))
- CountryCode (VARCHAR(255))
- HotelType (VARCHAR(100))
- HotelName (VARCHAR(255))


## Server IIT table to Vervotech mapping Table.

In [26]:
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError
from datetime import datetime
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Database credentials from .env
db_host = os.getenv('DB_HOST')
db_user = os.getenv('DB_USER')
db_pass = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')

# Create SQLAlchemy engine
SERVER_DATABASE_URL = f"mysql+pymysql://{db_user}:{db_pass}@{db_host}/{db_name}"
server_engine = create_engine(SERVER_DATABASE_URL)

def insert_data_in_chunks(engine, chunk_size=1000, page_size=1000):
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print("Starting data insertion process...")

    select_query = text("""
        SELECT * FROM innova_hotels_main
        WHERE SupplierCode =  'ratehawkhotel'
        LIMIT :limit OFFSET :offset
    """)

    insert_query = text("""
        INSERT INTO vervotech_mapping (
            last_update, VervotechId, ProviderHotelId, ProviderFamily, status,
            ModifiedOn, hotel_city, hotel_name, hotel_country, hotel_longitude,
            hotel_latitude, country_code, content_update_status, created_at
        )
        VALUES (:last_update, :VervotechId, :ProviderHotelId, :ProviderFamily, :status,
                :ModifiedOn, :hotel_city, :hotel_name, :hotel_country, :hotel_longitude,
                :hotel_latitude, :country_code, :content_update_status, :created_at)
    """)

    try:
        with engine.connect() as connection:
            offset = 0
            chunk_count = 1

            while True:
                # Fetch a small batch of data at a time
                result = connection.execute(select_query, {'limit': page_size, 'offset': offset}).fetchall()

                if not result:
                    print("No more records to transfer.")
                    break

                print(f"Fetched {len(result)} records from the database.")
                offset += page_size

                data_to_insert = []
                for row in result:
                    data_to_insert.append({
                        'last_update': current_time,
                        'VervotechId': None,
                        'ProviderHotelId': row[4], 
                        'ProviderFamily': row[3], 
                        'status': "Update",
                        'ModifiedOn': current_time,
                        'hotel_city': row[6],  
                        'hotel_name': row[14],
                        'hotel_country': row[11],  
                        'hotel_longitude': row[16],
                        'hotel_latitude': row[15], 
                        'country_code': row[12], 
                        'content_update_status': "Done",
                        'created_at': current_time  
                    })

                if data_to_insert:
                    try:
                        # Execute the bulk insert
                        connection.execute(insert_query, data_to_insert)
                        print(f"Chunk {chunk_count} inserted successfully.")
                        data_to_insert.clear()
                        chunk_count += 1
                        # Commit the transaction after each chunk
                        connection.commit()
                    except SQLAlchemyError as e:
                        print(f"Error during insert operation for chunk {chunk_count}: {e}")
                        connection.rollback()

    except SQLAlchemyError as e:
        print(f"Error during database operation: {e}")

    print("Data insertion completed.")

# Call the function to process data in chunks
insert_data_in_chunks(server_engine, chunk_size=1000, page_size=1000)


Starting data insertion process...
Fetched 1000 records from the database.
Chunk 1 inserted successfully.
Fetched 1000 records from the database.
Chunk 2 inserted successfully.
Fetched 1000 records from the database.
Chunk 3 inserted successfully.
Fetched 1000 records from the database.
Chunk 4 inserted successfully.
Fetched 1000 records from the database.
Chunk 5 inserted successfully.
Fetched 1000 records from the database.
Chunk 6 inserted successfully.
Fetched 1000 records from the database.
Chunk 7 inserted successfully.
Fetched 1000 records from the database.
Chunk 8 inserted successfully.
Fetched 1000 records from the database.
Chunk 9 inserted successfully.
Fetched 1000 records from the database.
Chunk 10 inserted successfully.
Fetched 1000 records from the database.
Chunk 11 inserted successfully.
Fetched 1000 records from the database.
Chunk 12 inserted successfully.
Fetched 1000 records from the database.
Chunk 13 inserted successfully.
Fetched 1000 records from the database